In [1]:
import credentials
credentials.credentials

{'City': {'name': 'Portland',
  'uaa': 'https://auth.aa.cityiq.io',
  'metadata': 'https://portland.cityiq.io/api/v2/metadata/',
  'event': 'https://portland.cityiq.io/api/v2/event/',
  'websocket': 'wss://portland.cityiq.io/api/v2/websocket',
  'developer': 'a1bd908f-1930-abe2-01db-20c3d34053e8:y7pFhvPx6ixYDpJ',
  'parking': 'Portland-IE-PARKING',
  'environment': 'Portland-IE-ENVIRONMENTAL',
  'pedestrian': 'Portland-IE-PEDESTRIAN',
  'traffic': 'Portland-IE-TRAFFIC',
  'bicycle': 'Portland-IE-BICYCLE',
  'bbox': '45.522295:-122.654883,45.49452:-122.537388'}}

In [2]:
import pandas as pd
import geopandas as gpd
from cityiq import CityIq
from cartoframes.auth import set_default_credentials
import time

In [3]:
set_default_credentials('creds_usignite.json')

In [6]:
myCIQ = CityIq("City")
myCIQ.fetchToken()
myCIQ.fetchMetadata("assets", "traffic", "eventTypes:TFEVT", size=100000)
vehicle_sensor_metadata = myCIQ.getAssets()

In [7]:
vehicle_sensor_metadata_df = pd.DataFrame(vehicle_sensor_metadata)
vehicle_sensor_metadata_df.dropna(inplace = True)
# split coordinates into lat and lng 
latlng = vehicle_sensor_metadata_df["coordinates"].str.split(":", n = 1, expand = True) 
vehicle_sensor_metadata_df["latitude"]= latlng[0].astype(float)
vehicle_sensor_metadata_df["longitude"]= latlng[1].astype(float)

### Set hours and end_time

In [9]:
# How many hours you want to request in total
h = 1
# When is the end hour you want to request
end_time = time.strptime('May 31 2020 10:00AM', '%b %d %Y %I:%M%p')

In [13]:
vehicle_hourly_aggregates = pd.DataFrame()
Time = int(time.mktime(end_time))*1000
# loop through assets to fetch events for each asset
for index, row in vehicle_sensor_metadata_df.iterrows():
    endTime = Time
    # Because when hours exceed 12, the server will be overloaded
    
    hours = 0
    
    while hours < h:
        # empty list to collect events
        vehicle_sensor_events_list = []
        hours = hours + 1
        
        startTime = endTime - 3600000 # startTime is 1 hour before endTime
        
        myCIQ.fetchEvents("assets", row.assetUid, "TFEVT", startTime, endTime, pageSize=100000)
        assetEvents = myCIQ.getEvents()
        
        
        
        if assetEvents == []:
            continue

        for a in assetEvents:
            a["directionUnit"] = a["properties"]["directionUnit"]
            a["speedUnit"] = a["properties"]["speedUnit"]
            a["eventUid"] = a["properties"]["eventUid"]
            a["counter_direction_speed"] = a["measures"]["counter_direction_speed"]
            a["counter_direction_vehicleCount"] = a["measures"]["counter_direction_vehicleCount"]
            a["vehicleCount"] = a["measures"]["vehicleCount"]
            a["counter_direction"] = a["measures"]["counter_direction"]
            a["speed"] = a["measures"]["speed"]
            a["direction"] = a["measures"]["direction"]
            vehicle_sensor_events_list.append(a)
        
        # with one hour of data make a dataframe, drop nulls, and group/aggregate vehicle counts 
        vehicle_sensor_events_df = pd.DataFrame(vehicle_sensor_events_list)
        vehicle_sensor_events_df.dropna(inplace = True)
        
        # group by location ID to get a sum of vehicleCounts 
        grouped_SD_veh_sensor_events_df = vehicle_sensor_events_df.groupby('assetUid').agg({'vehicleCount': ['sum'], 'counter_direction_vehicleCount': ['sum']})
        grouped_SD_veh_sensor_events_df['vehicleCount_sum'] = grouped_SD_veh_sensor_events_df['vehicleCount'] + grouped_SD_veh_sensor_events_df['counter_direction_vehicleCount']
        grouped_SD_veh_sensor_events_df = grouped_SD_veh_sensor_events_df.drop(['vehicleCount', 'counter_direction_vehicleCount'], axis=1)
        grouped_SD_veh_sensor_events_df['startTime'] = startTime
        grouped_SD_veh_sensor_events_df['assetUid'] = grouped_SD_veh_sensor_events_df.index
        grouped_SD_veh_sensor_events_df["latitude"] = row.latitude
        grouped_SD_veh_sensor_events_df["longitude"] = row.longitude
        
        vehicle_hourly_aggregates = pd.concat([vehicle_hourly_aggregates, grouped_SD_veh_sensor_events_df], ignore_index=True)
        
        endTime = startTime
    print("one iter finished!")

one iter finished!
one iter finished!
one iter finished!
one iter finished!
one iter finished!
one iter finished!
one iter finished!
one iter finished!
one iter finished!
one iter finished!
one iter finished!
one iter finished!
one iter finished!
one iter finished!
one iter finished!
one iter finished!
one iter finished!
one iter finished!
one iter finished!
one iter finished!
one iter finished!
one iter finished!
one iter finished!
one iter finished!
one iter finished!
one iter finished!
one iter finished!
one iter finished!
one iter finished!
one iter finished!
one iter finished!
one iter finished!
one iter finished!
one iter finished!
one iter finished!
one iter finished!
one iter finished!
one iter finished!
one iter finished!
one iter finished!
one iter finished!
one iter finished!
one iter finished!
one iter finished!
one iter finished!
one iter finished!
one iter finished!
one iter finished!
one iter finished!
one iter finished!
one iter finished!
one iter finished!
one iter fin

In [14]:
time_list = []
for i, row in vehicle_hourly_aggregates.iterrows():
    time_list.append(time.asctime(time.localtime(float(row['startTime']/1000))))
    
vehicle_hourly_aggregates['time'] = time_list
vehicle_hourly_aggregates = vehicle_hourly_aggregates.drop('startTime', axis=1)
vehicle_hourly_aggregates['vehicleCount_sum'] = vehicle_hourly_aggregates['vehicleCount_sum'].astype(int)

In [15]:
vehicle_hourly_aggregates.head(30)

,vehicleCount_sum,assetUid,latitude,longitude,time
,,,,,
0,89,01ec53e2-8464-4d0b-b0d4-76e1904d9af7,45.505296,-122.600155,Sun May 31 09:00:00 2020
1,105,042dd578-df60-443d-866f-3b985f881449,45.505321,-122.585876,Sun May 31 09:00:00 2020
2,92,05f04e82-9e3a-4d81-934b-6ba4435c1633,45.505299,-122.599534,Sun May 31 09:00:00 2020
3,60,0735adaf-c6ea-4ae7-9e8c-ee64e1779c02,45.504731,-122.621527,Sun May 31 09:00:00 2020
4,303,13aeaa1d-14ec-44f8-8747-0100817c74bc,45.504492,-122.562557,Sun May 31 09:00:00 2020
5,115,1b278267-564d-442d-95f7-b7637208263a,45.505321,-122.584632,Sun May 31 09:00:00 2020
6,100,1c303196-acd3-451b-8204-c97b5cd9c6fd,45.505307,-122.595183,Sun May 31 09:00:00 2020
7,56,20fc0a79-49f1-44eb-ba4e-7a49066ad21c,45.504745,-122.637547,Sun May 31 09:00:00 2020
8,60,225f070b-131f-4d0d-8195-4805d4ea78ff,45.512107,-122.646689,Sun May 31 09:00:00 2020


In [ ]:
vehicle_hourly_aggregates.to_csv("vehicle_count_sample.csv")